# 02 positional value Notebook

This notebook implements the analysis for the 02 positional value stage of the Fantasy Football Analysis project.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the src directory to path
sys.path.append('..')

# Set display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# TODO: Implement 02 positional value analysis

In [ ]:
# Import project modules
from src.data.data_loader import load_config, load_all_data
from src.data.data_processor import load_csv_data
from src.analysis.position_value import (
    calculate_vorp,
    analyze_positional_scarcity,
    analyze_position_groups
)
from src.visualization.position_vis import (
    plot_vorp_by_position,
    plot_positional_scarcity,
    plot_position_group_comparison
)
from src.utils.validation import validate_analysis_output

# Set pandas display options
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [ ]:
# 1. Load configuration and processed data
config = load_config()
performance_df = load_csv_data(os.path.join(config['data_paths']['processed_data'], 'player_performance.csv'))

In [ ]:
# 2. Calculate Value Over Replacement Player (VORP)
baseline_values = config['analysis']['baseline_values']
vorp_df = calculate_vorp(performance_df, baseline_values)

# Display top players by VORP
print("Top 20 players by VORP:")
display(vorp_df.sort_values('VORP', ascending=False)[
    ['Player', 'FantPos', 'Team', 'Half_PPR', 'VORP', 'VORP_Per_Game', 'G']
].head(20))

In [ ]:
# 3. Analyze positional scarcity
scarcity_results = analyze_positional_scarcity(vorp_df)

In [ ]:
# Display tier analysis for each position
positions = ['QB', 'RB', 'WR', 'TE']
for pos in positions:
    tier_key = f"{pos}_tiers"
    diff_key = f"{pos}_rank_diffs"
    
    if tier_key in scarcity_results:
        print(f"\n{pos} Tier Analysis:")
        display(scarcity_results[tier_key])
    
    if diff_key in scarcity_results:
        print(f"\nTop 10 {pos} Rank Differentials:")
        display(scarcity_results[diff_key].head(10))

# Display position value adjustments
print("\nPosition Value Adjustment Factors:")
display(scarcity_results['position_value_adjustments'])

In [ ]:
# 4. Analyze position groups
group_results = analyze_position_groups(vorp_df)

# Display position group stats
for key, df in group_results.items():
    print(f"\n{key.replace('_', ' ').title()}:")
    display(df)

In [ ]:
# 5. Create visualizations
# VORP by position
plt.figure(figsize=(12, 8))
plot_vorp_by_position(vorp_df)
plt.tight_layout()
plt.savefig('../outputs/figures/vorp_by_position.png')
plt.show()

# Positional scarcity
plt.figure(figsize=(14, 10))
plot_positional_scarcity(scarcity_results)
plt.tight_layout()
plt.savefig('../outputs/figures/positional_scarcity.png')
plt.show()

# Position group comparison
plt.figure(figsize=(12, 8))
plot_position_group_comparison(group_results)
plt.tight_layout()
plt.savefig('../outputs/figures/position_group_comparison.png')
plt.show()


In [ ]:
# 6. Save processed data
vorp_df.to_csv(os.path.join(config['data_paths']['processed_data'], 'vorp_analysis.csv'), index=False)


In [ ]:
# Save scarcity analysis
for key, df in scarcity_results.items():
    df.to_csv(os.path.join(config['data_paths']['processed_data'], f"{key}.csv"), index=False)

# Save position group analysis
for key, df in group_results.items():
    df.to_csv(os.path.join(config['data_paths']['processed_data'], f"{key}.csv"), index=False)

print("Positional value analysis completed!")